# PDF->Page->Text

This notebook processes PDFs to TXT by splitting the page and performing OCR for each page of the PDF.

## Outline of the Workflow
* Mount the notebook to the Google Colab
* Import Libraries and packages
* Format params, search, and get files from Google Drive
* Convert pdf pages to jpeg files
* Convert jpeg files to txt (string); save them in output_file

In [ ]:
### Modify this cell only
top = '10yRl5iz9bLn_-gQI--Dr700DreeUS89K' #1LfNpes0IEmbxinpRI_iIVCZuQ1waaCVJ this is the folder ID for SecondarySources , inputfolder id
input_dir = '/content/drive/My Drive/AWCA/Colab_notebooks/OCR/OCR_PDF2Page/hundreds_docs/'  ##must start and end with a "/"
output_dir = '/content/drive/My Drive/AWCA/Colab_notebooks/OCR/OCR_PDF2Page/hundreds_docs_results_2/' ##must start and end with a "/"

In [ ]:
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
service = build('drive', 'v3')

In [ ]:
# Run this to mount the Notebook in your Google Drive account 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


One-time installations

In [ ]:
!pip3 install pdf2image
!apt-get install poppler-utils
!pip3 install pytesseract
!pip3 install pdf2image
!sudo apt-get install tesseract-ocr
!pip install PyPDF2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 31 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.12 [154 kB]
Fetched 154 kB in 0s (1,796 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 160983 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.62.0-2ubuntu2.12_amd64.deb ...
Unpacking poppler-utils (0.62.0-2ubuntu2.12) ...
Setting up poppler-utils (0.62.0-2ubuntu2.12) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
  Created wheel for pytesseract: filename=pytesseract-0.3.7-py2.py3-none-any.whl size=13945 sha256=baa95b8656f90d4b290e790bd68e3d541584ca1b99a8dbc16eb490fd0aff59d7
  Stored in dir

Imports

In [ ]:
import os
import json
import csv
import sys
from apiclient.discovery import build  # pip install google-api-python-client
from pdf2image import convert_from_path # the module can convert PDF to a PIL Image object
from PIL import Image
import pytesseract #OCR tool; recognizes the text embedded in images
import sys 
from PyPDF2 import PdfFileReader, PdfFileWriter #the classes can read PDFs, split PDFs, and extract data
from collections import Counter # track the number of occurence of a value
import pandas as pd
import PIL
import re
import random

Logging setup

In [ ]:
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(threadName)-10s %(message)s',)
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

### Formatting and searching the files

In [ ]:
def iterfiles(name=None, is_folder=None, parent=None, order_by='folder,name,createdTime'):  
    q = []
    if name is not None:
        q.append("name = '%s'" % name.replace("'", "\\'"))
    if is_folder is not None:
        q.append("mimeType %s '%s'" % ('=' if is_folder else '!=', FOLDER))
    if parent is not None:
        q.append("'%s' in parents" % parent.replace("'", "\\'"))
    fields = ['id', 'title', 'mimeType']
    params = {'pageToken': None, 'orderBy': order_by,
              'fields': 'kind, nextPageToken, incompleteSearch, files(id, name, mimeType, md5Checksum, webViewLink, createdTime, modifiedTime, size)'}
    if q:
        params['q'] = ' and '.join(q)
    while True:
        logger.debug('params {}'.format(params))
        response = service.files().list(**params).execute() # searches the files which match **params
        logger.debug('response {}'.format(response))        
        for f in response['files']:
            yield f
        try:
            params['pageToken'] = response['nextPageToken'] # changes the page token of the file to next page token
        except KeyError:
            return

### Search all the sub-directories and files inside the initial directory

Uses `iterfiles()`

In [ ]:
def walk(top='root', by_name=False, snapshot_file='stack_snapshot.json'):
    print("walk function is called")
  # Search all the sub-directories and files inside the initial directory
    try:
        with open(snapshot_file) as f:
            stack = json.loads(f.read()) #accepts a string, read the json file
    except FileNotFoundError:
        logger.info('no stack snapshot found')
        stack = None
    if not stack:
      # if reading the json file was unsuccessful, check the reasons
        if by_name:
            top, = iterfiles(name=top, is_folder=True) # top as name
        else:
            top = service.files().get(fileId=top).execute() #top as fileId
            if top['mimeType'] != FOLDER:
                raise ValueError('not a folder: %r' % top)
        stack = [([top['name']], top)]
    while stack:
      # if stack is not empty, noting down its information
        logger.info('stack size {}'.format(len(stack)))
        with open(snapshot_file, 'w') as file:
            file.write(json.dumps(stack))
        path, top = stack.pop()
        logger.info('dir {}'.format(path))
        dirs, files = is_file = [], []
        for f in iterfiles(parent=top['id']):
            is_file[f['mimeType'] != FOLDER].append(f)
        logger.info('subdirs {} files {}'.format(len(dirs), len(files)))
        yield path, top, dirs, files
        if dirs:
            logger.debug('dirs {}'.format(dirs))
            logger.debug('path {}'.format(path))            
            newstuff = [(path + [d['name']], d) for d in reversed(dirs)]
            logger.debug('extend: {}'.format(newstuff))
            stack.extend(newstuff)

### Gives information about a file, including its ID, md5, size, create/modify time. 

In [ ]:
def extract_metadata(node):
    md5Checksum = node.get('md5Checksum', 'None')
    size = node.get('size', 'None')
    mimeType = node.get('mimeType', 'None')
    webViewLink = node.get('webViewLink', 'None')
    id = node.get('id', 'None')
    createdTime = node.get('createdTime', 'None')
    modifiedTime = node.get('modifiedTime', 'None')
    name = node.get('name', 'None')
    return id, md5Checksum, size, mimeType, createdTime, modifiedTime

### OCR on a Directory

<br/>

#### *Main Idea*: 
Takes in a directory, creates a folder of pages for each PDF/document, and uses OCR to convert all pages to computer text (txt).

<br/>

#### *Imports*: 
* PdfFileReader / PdfFileWriter: Useful for reading and saving PDFs.
* Counter: Counts the number of each element and stores the each unique as a key with the frequency as dictionary values. Subclass of dict. Used to count frequency of words in a PDF.

<br/>

#### *Outer loop*: Reads in all pdfs from the input directory from `walk()`. Splits a pdf into individual pages which will each be run into the inner for loop. Gets specific information about a file using `extract_metadata` and saves that in a CSV. Creates folder for output if one does not already exist.
* convert_from_path: takes in a PDF and outputs an image for each page in the PDF
* img_count: index for inner for loop. Tracks page number for a pdf.

<br/>

#### *Inner `page` loop*: For each page, sets Flag to False. Runs OCR with `pytesseract.[FILENAME]` on the default orientation and all 90-degree rotations: 0°, 90°, 180°, 270°. If any orientation can be read, Flag is set to True and the text of the page is written to output. If none of the rotations generates a proper output, the page is not used.

<br/>



In [ ]:
def dict_common_sum(cntr):
  sum = 0
  for word, freq in cntr.most_common(2):
    sum += freq * 2
  for word, freq in cntr.most_common(5):
    sum += freq * 2
  for word, freq in cntr.most_common(10):
    sum += freq
  return sum

def big_loop(top, csvwriter, maxfiles):   

  fc = 0
  if os.path.isdir(output_dir) == False:
    os.mkdir(output_dir)
  for path, root, dirs, files in walk(top=top, by_name=False, snapshot_file=snapshot_file):
      print("Checkpoint_0.1")
      # note 'by_name' is currently not used
      dirpath = '/'.join(path)
      id, _, _, mimeType, createdTime, modifiedTime = extract_metadata(root)
      # for directories we use the md5Checksum field to hold the dir count
      md5Checksum = len(dirs) 
      # for directories we use the size field to hold the plain file count
      size = len(files)
      row = [id, md5Checksum, size, mimeType, createdTime, modifiedTime, dirpath]
      csvwriter.writerow(row)
      # we count each dir as a file for the purpose of maxfiles
      fc = fc + 1
      for pdf_index in range(len(files)):
        print("Checkpoint_0")
        #Here we conduct page split and put it in a folder
        folder_name = str(files[pdf_index]['id'])+'/' ## This is how google id of the pdf gets to be the folder's name of that pdf
        folder_name_failed = str(id)+'/failed_pages.txt'  ## failed paged are specified in terms of filename and pages
        pdf_numbers = len(files)
        file_name = input_dir[:-1]+'/'+files[pdf_index]['name']
        if file_name.endswith('.pdf') == False:
          continue
        if os.path.isdir(output_dir+folder_name) == False:
          print("New folder is created")
          os.mkdir(output_dir+folder_name)
        pages = convert_from_path(file_name, output_folder=output_dir+folder_name)  ### Problem Here
        page_ids = []

        '''
        # ocr_ori
        cnt0, cnt1, cnt2, cnt3 = Counter(), Counter(), Counter(), Counter()
        num_to_select = max(min(5, len(pages)), int(0.1 * len(pages)))
        test_pages = random.sample(pages, num_to_select)
        
        for page_idx, page in enumerate(test_pages):
          img_count = page_idx + 1
          print("image idx: ", img_count)
          save_name = output_dir+folder_name+"page_"+str(img_count)+".jpg"
          page.save(save_name, 'JPEG')
          for i in range(4):
            text = str(((pytesseract.image_to_string(Image.open(save_name)))))
            if (i == 0):
              cnt0.update(text.split())
            if (i == 1):
              cnt1.update(text.split())
            if (i == 2):
              cnt2.update(text.split())
            if (i == 3):
              cnt3.update(text.split())
            page2 = page.rotate(90*(i + 1))
            page2.save(save_name,  'JPEG')
        w_0, w_1, w_2, w_3 = dict_common_sum(cnt0), dict_common_sum(cnt1), dict_common_sum(cnt2), dict_common_sum(cnt3)
        top_orient = max(w_0, w_1, w_2, w_3)
        if w_0 == top_orient:
          k = -1
        if w_1 == top_orient:
          k = 0
        if w_2 == top_orient:
          k = 1
        if w_3 == top_orient:
          k = 2
        print("Orientation to be used" + str(k))
        print(cnt0)
        print(len(cnt0))
        print(w_0)
        print(cnt1)
        print(len(cnt1))
        print(w_1)
        print(cnt2)
        print(len(cnt2))
        print(w_2)
        print(cnt3)
        print(len(cnt3))
        print(w_3)
        '''
        k = 1

        ori_found = False
        for file in os.listdir(output_dir+folder_name):
          # Pytesseract orientation
          if ori_found == False:
            im = Image.open(os.path.join(output_dir+folder_name+file))
            print(pytesseract.image_to_osd(im))
            osd = pytesseract.image_to_osd(im)
            #'''
            angle = re.search('(?<=Rotate: )\d+', osd).group(0)
            print("angle: ", angle)
            #'''
            ori_found = True

          if (os.path.splitext(file)[1].lower() == '.ppm'):
            page_ids.append(file)
        output_error_file = output_dir +folder_name+ 'failed_pages.txt' #Page the are failed in ocr process specified to pdf name and page number
        max_process = 5
        if len(pages) < max_process:
          print("Checkpoint_1: ", len(pages))
          start_idx = 0
          end_idx = len(pages)-1
          process_and_save(pages, output_dir, folder_name, start_idx, end_idx, output_error_file, pdf_index, files, page_ids, angle, k)
        else:
          print("Checkpoint_2: ", len(pages))
          start_idx = 0
          end_idx = max_process - 1
          while (end_idx < len(pages)):
            process_and_save(pages, output_dir, folder_name, start_idx, end_idx, output_error_file, pdf_index, files, page_ids, angle, k)
            if end_idx == len(pages)-1:
              break
            start_idx = start_idx+max_process
            if end_idx+max_process >= len(pages):
              end_idx = len(pages)-1
            else:
              end_idx = end_idx+max_process

        # Deleting ppms
        for file in page_ids:
          os.remove(os.path.join(output_dir+folder_name+file))

In [ ]:
def process_and_save(pages, output_dir, folder_name, start_idx, end_idx, output_error_file, pdf_index, files, page_ids, angle, k):
  csv_text = []
  csv_id = []
  pg_number = []
  pdf_name = []
  pages = pages[start_idx: end_idx+1]
  for page_idx, page in enumerate(pages):
    img_count = start_idx + 1
    print("image idx: ", img_count)
    save_name = output_dir+folder_name+"page_"+str(img_count)+".jpg"

    # pytesseract orientation using angle
    # page = page.rotate(int(angle) + 180)

    # ocr_ori
    page = page.rotate(90*(k + 1))

    # page.save(save_name, 'JPEG')
    ##Save the images and csv. 
    flag = True ##Currently disabling the oritation checker. Replace with a better scheme
    if flag == False:
        f_error = open(output_error_file, "a")
        message = "Page " + str(img_count) + " failed to find the correct orientation.\n"
        f_error.write(message)
        f_error.close() 
    else:
        # converts images to strings
        print(page_idx + start_idx, page)
        file_name = page_ids[start_idx + page_idx]
        # pytesseract orientation / ocr_ori using angle -> requires same rotate as above
        text = pytesseract.image_to_string(Image.open(os.path.join(output_dir+folder_name, file_name)).rotate(90*(k + 1)))
        csv_text.append(text)
        csv_id.append(str(files[pdf_index]['id'])+"_"+str(img_count))
        pdf_name.append(files[pdf_index]['name'])
        pg_number.append(start_idx + page_idx + 1)
    df = pd.DataFrame({'page id':csv_id, 'page number':pg_number, 'name': pdf_name, 'page text':csv_text})
    print("csv saved at " + output_dir+folder_name+str(start_idx)+"_"+"page.csv")
    df.to_csv(output_dir+folder_name+str(start_idx)+"_"+"page.csv") 

### Sets up output CSV file and calls `big_loop()`

In [ ]:
def catalog_walk(top):    ## pass in folder link
  snapshot_file = '/snapshot2021'
  csvoptions = {'dialect': csv.excel}
 
  OUTFILE = open("/content/drive/My Drive/catalog_not_in_use.csv", "a")
  csvwriter = csv.writer(OUTFILE)
  maxfiles = 100
  big_loop(top, csvwriter, maxfiles)
  OUTFILE.close()

### Specifies input and output directories and calls `catalog_walk()`

In [ ]:
snapshot_file = '/snapshot30001.txt'
FOLDER = 'application/vnd.google-apps.folder'
catalog_walk(top)

2021-04-17 01:21:04,770 INFO MainThread no stack snapshot found


walk function is called


2021-04-17 01:21:05,047 INFO MainThread stack size 1
2021-04-17 01:21:05,054 INFO MainThread dir ['hundreds_docs']
2021-04-17 01:21:05,391 INFO MainThread subdirs 0 files 2


Checkpoint_0.1
Checkpoint_0
New folder is created
Page number: 0
Orientation in degrees: 0
Rotate: 0
Orientation confidence: 14.83
Script: Latin
Script confidence: 5.19

angle:  0
Checkpoint_2:  554
image idx:  1
0 <PIL.Image.Image image mode=RGB size=1653x2339 at 0x7F43D6BE5050>
csv saved at /content/drive/My Drive/AWCA/Colab_notebooks/OCR/OCR_PDF2Page/hundreds_docs_results_2/1GfXGuNQhegD8ViN1qqY3Q6_PkGRDLG2H/0_page.csv
image idx:  1
1 <PIL.Image.Image image mode=RGB size=1653x2339 at 0x7F43D6AA8D50>
csv saved at /content/drive/My Drive/AWCA/Colab_notebooks/OCR/OCR_PDF2Page/hundreds_docs_results_2/1GfXGuNQhegD8ViN1qqY3Q6_PkGRDLG2H/0_page.csv
image idx:  1
2 <PIL.Image.Image image mode=RGB size=1653x2339 at 0x7F43D6AB9C90>
csv saved at /content/drive/My Drive/AWCA/Colab_notebooks/OCR/OCR_PDF2Page/hundreds_docs_results_2/1GfXGuNQhegD8ViN1qqY3Q6_PkGRDLG2H/0_page.csv
image idx:  1
3 <PIL.Image.Image image mode=RGB size=1653x2339 at 0x7F43D6BE5050>
csv saved at /content/drive/My Drive/AWCA